## Intrinsic parameter for a single camera
Output: intrinsic matrix K and distortion coefficients D

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import glob
import sys

if '/Users/damianstone/Documents/Code/tennis-project/post-triangulation/' not in sys.path:
  sys.path.append('/Users/damianstone/Documents/Code/tennis-project/post-triangulation/')


In [1]:
def print_image(imread_img):
  img = cv.cvtColor(imread_img, cv.COLOR_BGR2RGB)
  plt.imshow(img)
  plt.axis('off')
  plt.show()